<a href="https://colab.research.google.com/github/Faraaztqureshi/DS-Unit-2-Linear-Models/blob/master/Faraaz_Qureshi_017_DS10_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# Split the data by date

In [3]:
## inspect the data looking for dates
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
## convert the object type 'created' column to date/time
df['created'] = df['created'].apply(pd.to_datetime)

In [0]:
## creat a new column so I can later sort by month
df['month_created'] = df['created'].dt.month

In [13]:
## check to see how many samples for each month
df['month_created'].value_counts()

6    16973
4    16217
5    15627
Name: month_created, dtype: int64

In [0]:
## now to seperate into two groups
## one for properties listed in April May
## one for properties listed in June 
train = df[df['month_created'] != 6]
test = df[df['month_created'] == 6]

In [20]:
## check the output of work above
df.shape, train.shape, test.shape

((48817, 35), (31844, 35), (16973, 35))

In [22]:
## I want to ensure I didn't lose any data points
## this checks that rows of train and test sum to rows of df
train.shape[0] + test.shape[0] == 48817

True

# The code below is scratch, I just left it in there to show what I had to go through to reach success

In [0]:
## get dog friendly and cat friendly into seperate dfs
dogs = train[train['dogs_allowed', 'cats_allowed'] == 1]
cats = train[train['cats_allowed'] == 1]

In [0]:
condition = [1]

In [0]:
## I anticipate there being duplicates so I want to address this
## It took me a long time to figure out how to get a df with my two conditions

a = train[train.dogs_allowed.isin(condition) & train.cats_allowed.isin(condition)]
a['dogs_allowed'].value_counts()
pet_friendly = a

In [63]:
a.shape

(14175, 35)

In [39]:
## I anticipate there being duplicates so I want to address this
dogs['dogs_allowed'].value_counts()
cats['cats_allowed'].value_counts()

1    15194
Name: cats_allowed, dtype: int64

In [48]:
pet_friendly.shape

(14175, 35)

In [49]:
df.shape

(48817, 35)

In [0]:
## df['pet_friendly'] = 
row_indexes =df[df['dogs_allowed'] == 1].index
row_indexes
df.loc[row_indexes,'pet_friendly']= 1

# Feature 1: Pet Friendly.  Apartments that will accept both dogs and cats

In [0]:
## create a function that will go through columns flag rows that meet the conditions
def conditions(df):
    if (df['dogs_allowed'] == 1) and (df['cats_allowed'] == 1):
        return 1
    else:
        return 0

In [0]:
## make a new column with the results of above function
df['pet_friendly'] = df.apply(conditions, axis=1)

In [64]:
df['pet_friendly'].value_counts()

0    27001
1    21816
Name: pet_friendly, dtype: int64

# Feature 2: High End Amenities

In [0]:
## I will operationally define high end amenities as a doorman, swimming pool and fitness center
def conditions(df):
    if (df['doorman'] == 1) and (df['swimming_pool'] == 1) and (df['fitness_center'] == 1):
        return 1
    else:
        return 0

In [0]:
df['high_end_amenities'] = df.apply(conditions, axis=1)

In [72]:
## check to see how many apartments fit this criterion
df['high_end_amenities'].value_counts()

0    46408
1     2409
Name: high_end_amenities, dtype: int64

I'm basically returning to the top here, because I want to split the data after making the new features.

In [0]:
df['created'] = df['created'].apply(pd.to_datetime)

In [0]:
df['month_created'] = df['created'].dt.month

In [0]:
## now to seperate into two groups
## one for properties listed in April May
## one for properties listed in June 
train = df[df['month_created'] != 6]
test = df[df['month_created'] == 6]

# 2 Feature Linear Regression

In [103]:
target = 'price'
y_train = train[target]
y_test = test[target]
y_pred

array([3432.92724612, 3432.92724612, 3432.92724612, ..., 3642.09515718,
       3642.09515718, 3432.92724612])

In [0]:
## seperate the target variable
features = ['pet_friendly', 'high_end_amenities']
X_test  = test[features]
X_train = train[features]

In [0]:
## get LinearRegression
from sklearn.linear_model import LinearRegression
import numpy as np

In [0]:
## instantiate the class
model = LinearRegression()

In [0]:
## fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)


In [123]:
## obtain mean absolute error 
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Train Error: {mae:.2f} Dollars')

Train Error: 1186.44 Dollars


In [101]:
# Mean baseline
print('Mean Baseline (using 0 features)')
mean_baseline = y_train.mean()
print(mean_baseline)

Mean Baseline (using 0 features)
3575.604007034292


In [99]:
## these numbers make sense 
## it will add more cost with the higher amenities
model.coef_

array([209.16791106, 993.35659867])

In [100]:
model.intercept_

3432.927246118899

In [109]:
y_test.shape, y_pred.shape

((16973,), (16973,))

In [110]:
mean_absolute_error(y_test, y_pred)

1186.4394636594566

In [117]:
model.score(X_test, y_pred)

1.0

In [0]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [129]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'My model has a {mape:.2f}% error')

My model has a 35.25% error


# Analysis

From the work I did I got a value of 1186 for the mean absolute error.  This means that the model I created was on average making predictions that were $1186 more than the true outcomes.  I would say that by selecting higher end amenities as a feature, I threw off the model.  I would conclude that there are better features that could be utilized in order to develop a better more accurate model. 